In [22]:
import simpy
import numpy as np

# Constants
CAPACITY = 200
BASE_PRICE = 25
SIM_DAYS = 30  # Number of days to simulate
HOURS_PER_DAY = 12  # Number of hours to simulate per day (8 AM to 8 PM)

# Arrival rates per day of the week (hour ranges and rates)
ARRIVAL_RATES = {
    'Sunday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Monday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Tuesday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Wednesday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Thursday': [(8, 12, 15), (12, 17, 50), (17, 20, 50)],
    'Friday': [(8, 12, 20), (12, 17, 70), (17, 20, 80)],
    'Saturday': [(8, 12, 20), (12, 17, 70), (17, 20, 80)]
}

# Tracking logs for 30 days, 12 hours/day
price_log = np.zeros((SIM_DAYS, HOURS_PER_DAY))
occupancy_log = np.zeros((SIM_DAYS, HOURS_PER_DAY))

# Global variables
parked_customers = 0
lost_customers_due_to_full = 0

def get_arrival_rate(day, hour, occupancy):
    """Return arrival rate for a specific day and hour, adjusted by occupancy."""
    for start, end, rate in ARRIVAL_RATES[day]:
        if start <= hour < end:
            if occupancy >= 0.8:
                return rate * 0.6  # Reduce rate when occupancy > 80%
            elif occupancy < 0.3:
                return rate * 1.2  # Increase rate when occupancy < 30%
            else:
                return rate
    return 0

def calculate_price(base_price, occupancy):
    """Calculate parking price based on occupancy."""
    if occupancy >= 0.8:
        return base_price * 1.5  # Increase price if occupancy > 80%
    elif occupancy < 0.3:
        return base_price * 0.8  # Decrease price if occupancy < 30%
    return base_price

def vehicle(env, parking_lot, parking_duration):
    """Simulate a vehicle trying to park."""
    global parked_customers, lost_customers_due_to_full

    with parking_lot.request() as request:
        yield request  # Request parking spot

        if parking_lot.count >= CAPACITY:
            lost_customers_due_to_full += 1
            return  # Lot is full

        parked_customers += 1  # Park the vehicle
        yield env.timeout(parking_duration)  # Vehicle stays parked for its duration
        parked_customers -= 1  # Vehicle leaves

def arrival_process(env, parking_lot, day_name, day_index):
    """Simulate vehicle arrivals for 12 hours per day."""
    global parked_customers

    for hour in range(8, 20):  # Loop through hours (8 AM to 8 PM)
        hour_index = hour - 8
        occupancy = parked_customers / CAPACITY
        price = calculate_price(BASE_PRICE, occupancy)
        arrival_rate = get_arrival_rate(day_name, hour, occupancy)

        # Log data for this hour
        price_log[day_index, hour_index] = price
        occupancy_log[day_index, hour_index] = occupancy

        # Print simulation state
        print(f"Day {day_index + 1} ({day_name}), Hour {hour}: "
              f"Price = ₹{price:.2f}, Occupancy = {occupancy:.2%}, Arrival Rate = {arrival_rate:.2f}")

        # Generate vehicles based on arrival rate
        if arrival_rate > 0:
            while True:
                inter_arrival_time = np.random.exponential(1 / arrival_rate)
                if inter_arrival_time > 60:  # Skip if arrival interval is too long
                    break
                parking_duration = np.random.exponential(60)  # Average 1 hour parking
                env.process(vehicle(env, parking_lot, parking_duration))
                yield env.timeout(inter_arrival_time)

        # Ensure time advances correctly
        yield env.timeout(60)  # Wait for 1 hour (simulation time)

def run_simulation():
    """Run the parking simulation for 30 days and 12 hours per day."""
    global parked_customers, lost_customers_due_to_full
    for day_index in range(SIM_DAYS):
        day_name = list(ARRIVAL_RATES.keys())[day_index % 7]  # Cycle through week
        env = simpy.Environment()  # Create new environment for each day
        parking_lot = simpy.Resource(env, capacity=CAPACITY)

        # Reset global variables
        parked_customers = 0
        lost_customers_due_to_full = 0

        print(f"\nSimulating Day {day_index + 1} ({day_name})")
        env.process(arrival_process(env, parking_lot, day_name, day_index))
        env.run(until=HOURS_PER_DAY * 60)  # Run simulation for 12 hours (720 minutes)

# Run the simulation
run_simulation()



Simulating Day 1 (Sunday)
Day 1 (Sunday), Hour 8: Price = ₹20.00, Occupancy = 0.00%, Arrival Rate = 18.00

Simulating Day 2 (Monday)
Day 2 (Monday), Hour 8: Price = ₹20.00, Occupancy = 0.00%, Arrival Rate = 18.00

Simulating Day 3 (Tuesday)
Day 3 (Tuesday), Hour 8: Price = ₹20.00, Occupancy = 0.00%, Arrival Rate = 18.00

Simulating Day 4 (Wednesday)
Day 4 (Wednesday), Hour 8: Price = ₹20.00, Occupancy = 0.00%, Arrival Rate = 18.00

Simulating Day 5 (Thursday)
Day 5 (Thursday), Hour 8: Price = ₹20.00, Occupancy = 0.00%, Arrival Rate = 18.00

Simulating Day 6 (Friday)
Day 6 (Friday), Hour 8: Price = ₹20.00, Occupancy = 0.00%, Arrival Rate = 24.00

Simulating Day 7 (Saturday)
Day 7 (Saturday), Hour 8: Price = ₹20.00, Occupancy = 0.00%, Arrival Rate = 24.00

Simulating Day 8 (Sunday)
Day 8 (Sunday), Hour 8: Price = ₹20.00, Occupancy = 0.00%, Arrival Rate = 18.00

Simulating Day 9 (Monday)
Day 9 (Monday), Hour 8: Price = ₹20.00, Occupancy = 0.00%, Arrival Rate = 18.00

Simulating Day 10 (T